# Module 3: Feature selection - Practice

In this session you will practice **feature selection**, which reduces the dimensionality of data for the following reasons:

1. Reduces overfitting by removing noise introduced by some of the features.
2. Reduces training time, which allows you to experiment more with different models and hyperparameters.
3. Reduces data acquisition requirements.
4. Improves comprehensibility of the model because a smaller set of features is more comprehendible to humans. That will enable you to focus on the main sources of predictability, make the model more justifiable to another person.

We are going to use **titanic** dataset for this practice.

sklearn API reference:

+ [sklearn.feature_selection.SelectKBest](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html)
+ [sklearn.feature_selection.chi2](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.chi2.html)
+ [sklearn.preprocessing.LabelBinarizer](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelBinarizer.html)
+ [sklearn.feature_selection.sklearn.feature_selection.f_regression](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html)
+ [sklearn.preprocessing.scale](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html)
+ [sklearn.feature_selection.mutual_info_classif](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html)
+ [sklearn.feature_selection.RFE](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)


In [ ]:
import os, sys
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import RFE
from sklearn.base import clone

np.random.seed(18937)

## Load Dataset

In [ ]:
# Dataset location
DATASET = '/dsa/data/all_datasets/titanic_ML/titanic.csv'
assert os.path.exists(DATASET)

# Load and shuffle
dataset = pd.read_csv(DATASET).sample(frac = 1).reset_index(drop=True)
dataset.describe()

Create variable **X** and **y** and pull features and labels respectively.

In [ ]:
# Complete code below this comment (Question #P001)
# ----------------------------------
X = <placeholder>
y = <placeholder>

Create a train/validate split with **20%** of data held out for validation only.

In [ ]:
# Complete code below this comment (Question #P002)
# ----------------------------------
X_train, X_test, y_train, y_test = <placeholder>

# 1. Filter Methods

## A. χ² feature selection

Create a k-best feature selector with [SelectKBest](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html)
that uses [χ² feature selection](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.chi2.html)
to select **top 5** features.

In [ ]:
# Complete code below this comment (Question #P003)
# ----------------------------------
selector = SelectKBest(<placeholder>)

**Fit** the selector to the **training dataset**.

Then **print** χ² statistic (the score for this selector).

In [ ]:
# Add code below this comment (Question #P004)
# ----------------------------------






What are the columns indices that just got selected by the feature selector?

**Print these indices.**

In [ ]:
# Add code below this comment (Question #P005)
# ----------------------------------






What are the names of these columns selected?
Does it logically make sense that these features got selected?

**Print name of columns selected.**

In [ ]:
# Add code below this comment (Question #P006)
# ----------------------------------





Call **selector.transform()** method to select those feature columns from dataset.

In [ ]:
# Complete code below this comment (Question #P007)
# ----------------------------------
X_train_selected = <placeholder>
X_test_selected = <placeholder>

Now **fit** a Gaussian Naive Bayes model with these selected features.

**Compute validation accuracy** using model.score().

In [ ]:
# Complete code below this comment (Question #P008)
# ----------------------------------
model = GaussianNB()
model.fit(<placeholder>, y_train)
model.score(<placeholder>, y_test)

How does this compare to a model trained without feature selection?

In [ ]:
# Complete code below this comment (Question #P009)
# ----------------------------------
model = GaussianNB()
model.fit(<placeholder>, y_train)
model.score(<placeholder>, y_test)

## B. Mutual information

Mutual information is generally considered a robust measure of dependence, coming from information theory.
It could apply to both regression and classification problems.
sklearn provided [mutual_info_classif()](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html#sklearn.feature_selection.mutual_info_classif)
and [mutual_info_regression()](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_regression.html#sklearn.feature_selection.mutual_info_regression) respectively.

We review feature selection again and also try this method below.

**Tip**: _Putting Python variables in a function allows you to limit its scope and hide global variables.
Only keywords that create new scope in Python are **class**, **def** and **lambda** ._

In [ ]:
# Complete code below this comment (Question #P010)
# ----------------------------------
def mutual_info_session():
    selector = SelectKBest(mutual_info_classif, k=3)
    selector.fit(<placeholder>, y_train)
    print(selector.get_support(True))
    model = GaussianNB()
    model.fit(<placeholder>, y_train)
    return model.score(selector.transform(X_test), y_test)
    
mutual_info_session()

# 2. Wrapper Methods 
## A. Forward selection

Complete following code based on your understanding of forward selection.
Please refer to the feature selection lab, same section for hints.

In [ ]:
class ForwardSelector(object):
    def __init__(self, estimator):
        self.estimator = estimator
        
    def fit(self, X, y, k):
        selected = np.zeros(X.shape[1]).astype(bool)
        score = lambda X_features: clone(self.estimator).fit(X_features, y).score(X_features, y)
        selected_indices = lambda: list(np.flatnonzero(selected))

        # What is the exit condition for forward selection?
        # ---------------------------------- (Question #P3011)
        while <placeholder>:
            rest_indices = list(np.flatnonzero(~selected))
            
            scores = list()
            for i in rest_indices:
                # Which columns are we currently using to score the model?
                # ---------------------------------- (Question #P3012)
                feature_subset = <placeholder>
                s = score(X[:, feature_subset])
                scores.append(s)
           
            idx_to_add = rest_indices[np.argmax(scores)]
            selected[idx_to_add] = True

        self.selected = selected.copy()
        return self
        
    def transform(self, X):
        return X[:, self.selected]
    
    def get_support(self, indices=False):
        return np.flatnonzero(self.selected) if indices else self.selected
    

Now **write a test case** for your completed forward selection algorithm and see if it works well for you.
**Select 3 features.**

**Tip**: _You could copy-paste-edit what we just did in mutual information section._

In [ ]:
def forward_selection_session():
    model = GaussianNB()
    selector = ForwardSelector(model)
    # Add code below this comment (Question #P013)
    # ----------------------------------




    # ----------------------------------
    return model.score(selector.transform(X_test), y_test)
    
forward_selection_session()

## B. Recursive feature elimination

Finally, also give it a shot at RFE provided by sklearn.

In [ ]:
# Complete code below this comment 
# ----------------------------------
def rfe_session():
    # using SVC instead because RFE doesn't support GaussianNB
    from sklearn.svm import SVC
    model = SVC(kernel="linear")
    # Add code below this comment (Question #P014)
    # ----------------------------------




    # ----------------------------------
    model.fit(selector.transform(X_train), y_train)
    
    return model.score(selector.transform(X_test), y_test)

rfe_session()

# Save your notebook!  Then `File > Close and Halt`